In [ ]:
# -*- coding: utf-8 -*-
import sys
import numpy as np
import codecs
import io
import os
import sys
import matplotlib.pyplot as plt
from urllib import request
import zipfile

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

In [ ]:
#reading fastText pre-trained vectors
word2vec_file = 'cc.en.300.vec'
word2vec_origin = 'https://s3-us-west-1.amazonaws.com/fasttext-vectors/cc.en.300.vec.gz'

In [ ]:
# this code from https://github.com/facebookresearch/fastText/blob/master/docs/english-vectors.md
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

word2vec = load_vectors('cc.en.300.vec')

In [ ]:
# PMSP96 データの読み込み
filename='../data/PMSP96.orig'
words, freqs, orthography, phonology, phonemes = {}, {}, {}, {}, {}
n = 0
with codecs.open(filename,'r') as f:
    for line in f:
        a = line.strip().split(' ')
        if n % 3 == 0:
            #if a[0] == '#':
            seq, word, phoneme, freq = a[1], a[2], a[3], a[4]
            if word in words:
                print(seq, word)
            words[word] = int(seq)
            freqs[word] = float(freq)
            phonemes[word] = phoneme
        elif n % 3 == 1:
            orthography[word] = np.asarray([int(x) for x in a])
        else:
            phonology[word] = np.asarray([int(x) for x in a])
        n += 1

print('vocab in orthography:{0}, phonology:{1}'.format(len(orthography), len(phonology)))

In [ ]:
semantics = {}
for word in orthography:
    if word in word2vec:
        semantics[word] = word2vec[word]
    else:
        print('"{0:s}" was not found in semantics'.format(word))
        
len(semantics)        

In [ ]:
prev_word = None
same_count = 0
for i, word in enumerate(orthography):
    #if not word in semantics1bill:
    #    print(i, word, 'in 1billion')
    if not word in semantics:
        print(i, word, 'in wiki-news')  #, wrd2idx[word], semantics[word])
        #del orthography[word]
        #del phonology[word]
    if prev_word == word:
        same_count += 1
    prev_word = word

In [ ]:
x_prev = -1
for word in words:
    x = words[word]
    if (x_prev + 1) != x:
        print(x, word)
    x_prev = x

In [ ]:
filename = 'pmsp96.pk'
pmsp96data = {'orthography':orthography, 
              'phonology':phonology, 
              'semantics':semantics,
              'words':words,
              'freqs':freqs,
              'phoneme':phonemes}
with codecs.open(filename,'wb') as f:
    pickle.dump(pmsp96data,f)